# Run it

To run it use

`docker run -v ${PWD}:/notebook -p 8888:8888 ermaker/keras-jupyter` 

## Data preparation



In [18]:

def load_sentences(file_path):
    from xml.etree import cElementTree as ET
    t = ET.parse(file_path)
    sentences = []
    for article in list(t.getroot()):
        cur = []
        for sentence in  list(article):
            for word in list(sentence):
                cur.append(word.text)
                
        sentences.append(cur)
        
    return sentences

In [23]:
blick = load_sentences('../data/NOAHsCorpusOfSwissGermanDialects_Release2.1/blick.xml')
blogs = load_sentences('../data/NOAHsCorpusOfSwissGermanDialects_Release2.1/blogs.xml')
schobinger = load_sentences('../data/NOAHsCorpusOfSwissGermanDialects_Release2.1/schobinger.xml')
swatch = load_sentences('../data/NOAHsCorpusOfSwissGermanDialects_Release2.1/swatch.xml')
wiki = load_sentences('../data/NOAHsCorpusOfSwissGermanDialects_Release2.1/wiki.xml')

all_sentences = blick + blogs + schobinger + swatch + wiki